![ThermoFun logo](../common/fun_long_logo-01.png)

**ThermoFun** is an open-source client that delivers thermodynamic properties of substances and reactions at the temperature and pressure of interest.

### Example for calculation the water density and dielectric constant

In [ ]:
import thermofun as fun
import numpy as np
import matplotlib.pyplot as plt

database = fun.Database('../databases/slop98-inorganic-thermofun.json') # load database containing data necessary to calculate properties 
database.appendData('water-thermofun.json')
engine = fun.ThermoEngine(database) # create an engine to have access to the functions for calculating properties

In [ ]:
engine.propertiesSolvent(400+273.15, 300e05, 'H2O@')

In [ ]:
rho_solvent = engine.propertiesSolvent(400+273.15, 300e05, 'H2O@')
e_solvent = engine.electroPropertiesSolvent(400+273.15, 300e05, 'H2O@')

In [ ]:
rho_solvent.density.val

In [ ]:
e_solvent.epsilon.val

In [ ]:
# functions to loop through P-T list and calculate water properties

# example for using two available database records, 
# one where the state of water is gas (H2Og ) is used to calculate the properties in the gas field  "aggregate_state": { "0": "AS_GAS" }
# and another where the state of water is aqueous (H2O@ ) is used to calculate the properties in the gas field  "aggregate_state": { "4": "AS_AQUEOUS" }
def water_density(T, P):
    try:
        if (T<647.067e0 ): # below crtical point check for liquid or vapor region
            p = engine.propertiesSolvent(T, 0, 'H2Og').pressure.val
            if (P<p and P>0): # in the vapor region
                #print (f"pcalc {p} pgiven {P}")
                return engine.propertiesSolvent(T, P, 'H2Og').density.val
        return engine.propertiesSolvent(T, P, 'H2O@').density.val
    except Exception as e:
        #print(f"Error calculating density for T={T}, P={P}: {e}")
        return 1000

def water_pressure(T, P):
    try:
        return engine.propertiesSolvent(T, P, 'H2O@').pressure.val
    except Exception as e:
        #print(f"Error calculating density for T={T}, P={P}: {e}")
        return 0


def water_epsilon(T, P):
    try:
        if (T<647.067e0 ): # below crtical point check for liquid or vapor region
            p = engine.propertiesSolvent(T, 0, 'H2Og').pressure.val
            if (P<p and P>0): # in the vapor region
                #print (f"pcalc {p} pgiven {P}")
                return engine.electroPropertiesSolvent(T, P, 'H2Og').epsilon.val
        return engine.electroPropertiesSolvent(T, P, 'H2O@').epsilon.val
    except Exception as e:
        #print(f"Error calculating dielectric constant for T={T}, P={P}: {e}")
        return 100

# example for giving state of water explicitly as last function argument 0-liquid, 1-gas/vapor
# is sufficient to only use H2O@ record from the database
def water_density2(T, P):
    try:
        if (T<647.067e0 ): # below crtical point check for liquid or vapor region
            p = engine.propertiesSolvent(T, 0, 'H2O@').pressure.val
            if (P<p and P>0): # in the vapor region
                #print (f"pcalc {p} pgiven {P}")
                return engine.propertiesSolvent(T, P, 'H2O@', 1).density.val
        return engine.propertiesSolvent(T, P, 'H2O@', 0).density.val
    except Exception as e:
        #print(f"Error calculating density for T={T}, P={P}: {e}")
        return 1000




# give state as last function argument 0-liquid, 1-gas/vapor
def water_epsilon2(T, P):
    try:
        if (T<647.067e0 ): # below crtical point check for liquid or vapor region
            p = engine.propertiesSolvent(T, 0, 'H2O@').pressure.val
            if (P<p and P>0): # in the vapor region
                #print (f"pcalc {p} pgiven {P}")
                return engine.electroPropertiesSolvent(T, P, 'H2O@', 1).epsilon.val
        return engine.electroPropertiesSolvent(T, P, 'H2O@', 0).epsilon.val
    except Exception as e:
        #print(f"Error calculating dielectric constant for T={T}, P={P}: {e}")
        return 100



In [ ]:
# Define the temperature and pressure intervals
T_min, T_max, T_steps = 0, 600, 200  # Example: from 0 to 100 degrees Celsius
P_min, P_max, P_steps = 1, 500, 100  # Example: from 1 to 100 bars

# Generate temperature and pressure ranges
T_values = np.linspace(T_min, T_max, T_steps)
P_values = np.linspace(P_min, P_max, P_steps)

# Create a 2D array to store density values
density_values = np.zeros((P_steps, T_steps))

# Calculate the density for each T and P
for i, T in enumerate(T_values):
    for j, P in enumerate(P_values):
        d = water_density2(T+273.15, P*1e5)
        #if P == 0.0 :
        #    P_values[j]=water_pressure(T+273.15, P*1e5)/1e5
        if (d == 0):
            print(f"zero density for T={T-273.15}, P={P/1e5}")
        density_values[j, i] = d

# Create the plot
plt.figure(figsize=(10, 8))
T_mesh, P_mesh = np.meshgrid(T_values, P_values)
contour = plt.contourf(T_mesh, P_mesh, density_values, 1000, cmap='viridis')

plt.colorbar(contour)
plt.title('Water Density as a Function of Temperature and Pressure')
plt.xlabel('Temperature (C)')
plt.ylabel('Pressure (bar)')

plt.show()


In [ ]:
if np.any(P_values == 0):
    print("The matrix contains zero.")
else:
    print("The matrix does not contain zero.")

In [ ]:

if np.any(density_values == 0):
    print("The matrix contains zero.")
else:
    print("The matrix does not contain zero.")

In [ ]:
# Create a 2D array to store density values
epsilon_values = np.zeros((P_steps, T_steps))

# Calculate the density for each T and P
for i, T in enumerate(T_values):
    for j, P in enumerate(P_values):
        epsilon_values[j, i] = water_epsilon2(T+273.15, P*1e5)

# Create the plot
plt.figure(figsize=(10, 8))
T_mesh, P_mesh = np.meshgrid(T_values, P_values)
contour = plt.contourf(T_mesh, P_mesh, epsilon_values,100, cmap='viridis')

plt.colorbar(contour)
plt.title('Water Dielectric constant as a Function of Temperature and Pressure')
plt.xlabel('Temperature (C)')
plt.ylabel('Pressure (bar)')

plt.show()

In [ ]:
# Create lists to store the data
T_list = []
P_list = []
density_list = []

# Calculate the density for each T and P
for T in T_values:
    for P in P_values:
        density = water_density(T+273.15, P*1e5)
        T_list.append(T)
        P_list.append(P)
        density_list.append(density)

# Convert lists to numpy arrays
T_array = np.array(T_list)
P_array = np.array(P_list)
density_array = np.array(density_list)

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the data
sc = ax.scatter(T_array,  P_array, density_array, c=density_array, cmap='viridis')

# Add color bar
cbar = plt.colorbar(sc)
cbar.set_label('Density')

# Set labels
ax.set_xlabel('Temperature (°C)')
ax.set_zlabel('Density')
ax.set_ylabel('Pressure (bar)')
ax.set_title('Water Density as a Function of Temperature and Pressure')

plt.show()

In [ ]:
# Create lists to store the data
T_list = []
P_list = []
epsilon_list = []

# Calculate the density for each T and P
for T in T_values:
    for P in P_values:
        epsilon = water_epsilon(T+273.15, P*1e5)
        T_list.append(T)
        P_list.append(P)
        epsilon_list.append(epsilon)

# Convert lists to numpy arrays
T_array = np.array(T_list)
P_array = np.array(P_list)
epsilon_array = np.array(epsilon_list)

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the data
sc = ax.scatter(T_array,  P_array, epsilon_array, c=epsilon_array, cmap='viridis')

# Add color bar
cbar = plt.colorbar(sc)
cbar.set_label('Epsilon')

# Set labels
ax.set_xlabel('Temperature (°C)')
ax.set_zlabel('Epsilon')
ax.set_ylabel('Pressure (bar)')
ax.set_title('Water Dielectric constant as a Function of Temperature and Pressure')

plt.show()

In [ ]:
# use a different EoS for Water
# "32": "water_eos_iapws95_reaktoro" Wagner and Pruss EoS
print(engine.propertiesSolvent(400+273.15, 300e05, 'H2O@reak').density.val)
print(engine.electroPropertiesSolvent(400+273.15, 300e05, 'H2O@reak').epsilon.val)